# CHIPPR

This notebook demonstrates the use of the Cosmological Hierarchical Inference with Probabilistic Photometric Redshifts (CHIPPR) package to estimate the redshift density function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from chippr import *

## Simulation

## Inference

In [ ]:
data_loc = '../../../prob-z/nz/code/tests/sig2/data/logdata.csv'

with open(data_loc,'rb') as data_file:
    tuples = (line.split(None) for line in data_file)
    data = [[float(pair[k]) for k in range(0,len(pair))] for pair in tuples]

In [ ]:
zs = np.array(data[0])
nz_intp = np.array(data[1])
z_posts = np.array(data[2:])

print(np.shape(zs), np.shape(nz_intp), np.shape(z_posts))

z_mids = (zs[1:]+zs[:-1])/2.
n_bins = len(z_mids)

In [ ]:
prior_sigma = 0.16
prior_var = np.eye(n_bins)
for b in range(nbins):
    prior_var[b] = 1. * np.exp(-0.5 * (z_mids[b] - z_mids) ** 2 / prior_sigma ** 2)
l = 1.e-4
prior_var = prior_var+l*np.identity(n_bins)

In [ ]:
nz = z_dens(data, prior_var, vb=True)

In [ ]:
help(z_dens)

In [ ]:
nz_mmap = nz.mmap()

In [ ]:
nz_mexp = nz.mexp()

In [ ]:
nz_stack = nz.stack()

In [ ]:
nz_mmle = nz.infer(nz_lf)

In [ ]:
nz_samps = nz.sample(nz_lf)

In [ ]:
nz.plot()